# Exective summary of Work Package 2

## Objectives

In this WP, you will work on a given training dataset. Your goal is to develop a fault detection model using the classification algorithms learnt in the class, in order to achieve best F1 score.

## Tasks

- Task 1: Develop a fault detection model using the unsupervised learning algorithms learnt in the class, in order to achieve best F1 score.
- Task 2: With the help of the supporting script, develop a cross-validation scheme to test the performance of the developed classification algorithms.
- Task 3: Develop a fault detection model using the classification algorithms learnt in the class, in order to achieve best F1 score.

## Delierables

- A Jupyter notebook reporting the process and results of the above tasks


# Before starting, please:
- Fetch the most up-to-date version of the github repository.
- Create a new branch with your name, based on the "main" branch and switch to your own branch.
- Copy this notebook to the work space of your group, and rename it to TD_WP_2_Your name.ipynb
- After finishing this task, push your changes to the github repository of your group.

# Task 1: Unsupervised learning approaches

## Implement the statistical testing approach for fault detection

In this exercise, we interpret the statistical testing approach for fault detection. The basic idea of statistical testing approach is that we fit a multi-dimensitional distribution to the observation data under normal working condition. Then, when a new data point arrives, we design a hypothesis test to see whether the new data point is consistent with the distribution. If the new data point is consistent with the distribution, we can conclude that the fault is not due to the faulty component.

The benefit of this approach is that, to design the detection algrothim, we do not need failed data. Also, the computational time is short as all we need is just to compute the pdf and compare it to a threshold.

In this exercise, you need to:
- Fit a multi-dimensitional distribution to the training dataset (all normal samples).
- Design a fault detection algorithm based on the fitted distribution to detect faulty components.

The following block defines a few functions that you can use.

In [355]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal


def estimateGaussian(X):
    '''Given X, this function estimates the parameter of a multivariate Gaussian distribution.'''
    mu = np.mean(X, axis=0)
    sigma2 = np.var(X, axis=0)
    return mu, sigma2


def classify(X, distribution, log_epsilon=-50):
    '''Given X, this function classifies each sample in X based on the multivariate Gaussian distribution. 
       The decision rule is: if the log pdf is less than log_epsilon, we predict 1, as the sample is unlikely to be from the distribution, which represents normal operation.
    '''
    p = distribution.logpdf(X)
    predictions = (p < log_epsilon).astype(int)
    
    return predictions

Let us use the dataset `20240105_164214` as training dataset, as all the samples in this dataset are normal operation. We will use the dataset `20240325_155003` as testing dataset. Let us try to predict the state of motor 1. For this, we first extract the position, temperature and voltage of motor 1 as features (you can change the features if you want). 

In [356]:
import sys
sys.path.append('../supporting_scripts/WP_1')

from utility import read_all_csvs_one_test
import pandas as pd

# Specify path to the dictionary.
base_dictionary = '../dataset/training_data/'
dictionary_name = '20240105_164214'
path = base_dictionary + dictionary_name

# Read the data.
df_data = read_all_csvs_one_test(path, dictionary_name)

# Get the features
X_train = df_data[['data_motor_1_position', 'data_motor_1_temperature', 'data_motor_1_voltage']]

# We do the same to get the test dataset.
dictionary_name = '20240325_155003'
path = base_dictionary + dictionary_name

# Read the data.
df_data = read_all_csvs_one_test(path, dictionary_name)

# Get the features
X_test = df_data[['data_motor_1_position', 'data_motor_1_temperature', 'data_motor_1_voltage']]
y_test = df_data['data_motor_1_label']

Please design your algorithm below:

In [357]:
from sklearn.metrics import f1_score, accuracy_score

# First, we need to fit a MVN distribution to the normal samples.
mu, sigma2 = estimateGaussian(X_train)

# Construct a multivariate Gaussian distribution to represent normal operation.
distribution = multivariate_normal(mean=mu, cov=np.diag(sigma2))

# Now, let's try to predict the labels of the test set X_test.
y_pred = classify(X_test, distribution)

# Calculate accuracy of the prediction.
f1 = f1_score(y_test, y_pred)
print("F1-score:", f1)


F1-score: 0.3252769385699899


**Discussions:**
- Can you please try to improve the performance of this approach?
    - For example, by normalizating the data?
    - By smoothing the data?
    - By reducing feature number?
    - etc.
- The parameter log_epsilon defines the threshold we use for making classification. What happens if you change it?
- Could you discuss how we should get the best value for this parameter?

### Preprocessing

In [358]:
print(X_train.columns)

Index(['data_motor_1_position', 'data_motor_1_temperature',
       'data_motor_1_voltage'],
      dtype='object')


In [359]:
#Standardization : 
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
cols = X_train.columns
X_scaled = pd.DataFrame(scaler.fit_transform(X_train))
X_scaled.columns = cols


In [360]:
#Deal with outliers : 
for column in X_scaled.columns : 
    Q1 = X_scaled[column].quantile(0.25)
    Q3 = X_scaled[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    X_scaled[column]= np.where(X_scaled[column] > upper_bound, upper_bound, X_scaled[column])
    X_scaled[column] = np.where(X_scaled[column] < lower_bound, lower_bound, X_scaled[column])
    
X_scaled.describe()

,data_motor_1_position,data_motor_1_temperature,data_motor_1_voltage
count,3.652000e+03,3.652000e+03,3.652000e+03
mean,4.111436e-01,-3.113002e-16,9.214486e-15
std,1.110375e-16,1.000137e+00,1.000137e+00
min,4.111436e-01,-2.687957e+00,-2.233978e+00
25%,4.111436e-01,-8.184004e-01,-7.114257e-01
50%,4.111436e-01,4.279708e-01,-4.123530e-01
75%,4.111436e-01,1.051156e+00,1.083011e+00
max,4.111436e-01,1.051156e+00,2.469621e+00


In [361]:
cols = X_train.columns
X_train2 = pd.DataFrame(scaler.fit_transform(X_scaled))
X_train2.columns = cols
X_train2 = X_train2.drop("data_motor_1_position", axis = 1)

X_train

,data_motor_1_position,data_motor_1_temperature,data_motor_1_voltage
0,86,42,7223
1,86,42,7214
2,86,42,7137
3,86,42,7135
4,86,42,7212
...,...,...,...
3647,86,48,7144
3648,86,48,7148
3649,86,48,7241
3650,86,48,7228


In [362]:
# First, we need to fit a MVN distribution to the normal samples.
mu, sigma2 = estimateGaussian(X_train2)

# Construct a multivariate Gaussian distribution to represent normal operation.
distribution2 = multivariate_normal(mean=mu, cov=np.diag(sigma2), allow_singular=True)

# Now, let's try to predict the labels of the test set X_test.
y_pred_preprocess = classify(X_test, distribution2, log_epsilon=-2.5*10**7)

# Calculate accuracy of the prediction.
accuracy = accuracy_score(y_test, y_pred_preprocess)
f1 = f1_score(y_test, y_pred_preprocess)

print("Accuracy:", accuracy)
print("F1-score:", f1)


ValueError: operands could not be broadcast together with shapes (6652,3) (2,) 

In [363]:
from sklearn.metrics import accuracy_score

def dichotomy_algorithm(X_test, y_test, distribution, log_epsilon_min, log_epsilon_max, tol=0.1):
    # Define a tolerance level to stop the iteration
    diff = log_epsilon_max - log_epsilon_min
    eps = (log_epsilon_max + log_epsilon_min) / 2

    # Iteratively adjust log_epsilon until convergence
    while diff > tol:       
        y_pred_eps = classify(X_test, distribution, log_epsilon = eps)
        accuracy = f1_score(y_test, y_pred_eps) 

        y_pred_ = classify(X_test, distribution, log_epsilon = log_epsilon_min)
        accuracy_min = f1_score(y_test, y_pred_preprocess)
        
        # Update log_epsilon_min or log_epsilon_max based on the accuracy
        if accuracy_min < accuracy:
            log_epsilon_min = eps
        else:
            log_epsilon_max = eps
        
        eps = (log_epsilon_max + log_epsilon_min) / 2
        # Update the difference for convergence check
        diff = log_epsilon_max - log_epsilon_min
        print(eps, accuracy)
    
    # Return the optimal log_epsilon_mid
    return eps, accuracy

# Set initial values for log_epsilon_min and log_epsilon_max
log_epsilon_min = -10**10  # Initial lower bound
log_epsilon_max = 0   # Initial upper bound

# Call the dichotomy_algorithm function
optimal_log_epsilon, optimal_accuracy = dichotomy_algorithm(X_test, y_test, distribution, log_epsilon_min, log_epsilon_max)

# Print the optimal values
print("Optimal log_epsilon:", optimal_log_epsilon)
print("Corresponding accuracy:", optimal_accuracy)

-7500000000.0 0.0
-8750000000.0 0.0
-9375000000.0 0.0
-9687500000.0 0.0
-9843750000.0 0.0
-9921875000.0 0.0
-9960937500.0 0.0
-9980468750.0 0.0
-9990234375.0 0.0
-9995117187.5 0.0
-9997558593.75 0.0
-9998779296.875 0.0
-9999389648.4375 0.0
-9999694824.21875 0.0
-9999847412.109375 0.0
-9999923706.054688 0.0
-9999961853.027344 0.0
-9999980926.513672 0.0
-9999990463.256836 0.0
-9999995231.628418 0.0
-9999997615.814209 0.0
-9999998807.907104 0.0
-9999999403.953552 0.0
-9999999701.976776 0.0
-9999999850.988388 0.0
-9999999925.494194 0.0
-9999999962.747097 0.0
-9999999981.373549 0.0
-9999999990.686775 0.0
-9999999995.343388 0.0
-9999999997.671694 0.0
-9999999998.835846 0.0
-9999999999.417923 0.0
-9999999999.708961 0.0
-9999999999.85448 0.0
-9999999999.92724 0.0
-9999999999.96362 0.0
Optimal log_epsilon: -9999999999.96362
Corresponding accuracy: 0.0


In [ ]:
#Dichotomy for epsilon : 

In [ ]:
print(y_pred.mean())
print(y_pred_preprocess.mean())

1.0
0.06584485868911606


## Local outiler factor (LOF)

The local outlier factor (LOF) algorithm computes the local density deviation of a given data point with respect to its neighbors. It considers as outliers the samples that have a substantially lower density than their neighbors. You can easiliy implement LOF in scikit-learn ([tutorial](https://www.datatechnotes.com/2020/04/anomaly-detection-with-local-outlier-factor-in-python.html)).

Please implement local outlier factor (LOF) algorithm on the dataset of `20240325_155003`. You can try first to detect the failure of motor 1 using this model. Please calculate the accuracy score of your prediction.